SPOTIFY EDA

In [1]:
import pandas as pd

import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [51]:
# artist_df = pd.read_csv('songs_dataset/data_by_artist.csv')
artist_df = pd.read_csv('songs_dataset/data.csv', delim_whitespace=True)
artist_df[artist_df.artists]

KeyError: "None of [Index(['['Carl Woitschach']', '['Robert Schumann', 'Vladimir Horowitz']',\n       '['Seweryn Goszczyński']', '['Francisco Canaro']',\n       '['Frédéric Chopin', 'Vladimir Horowitz']',\n       '['Felix Mendelssohn', 'Vladimir Horowitz']',\n       '['Franz Liszt', 'Vladimir Horowitz']', '['Carl Woitschach']',\n       '['Francisco Canaro', 'Charlo']', '['Seweryn Goszczyński']',\n       ...\n       '['YoungBoy Never Broke Again']', '['Kelly Clarkson']',\n       '['Meek Mill', 'Roddy Ricch']', '['Trey Songz', 'Summer Walker']',\n       '['LEGADO 7', 'Junior H']', '['DripReport', 'Tyga']',\n       '['Leon Bridges', 'Terrace Martin']', '['Kygo', 'Oh Wonder']',\n       '['Cash Cash', 'Andy Grammer']', '['Ingrid Andress']'],\n      dtype='object', length=169909)] are in the [columns]"

In [25]:
# INITIALIZE SPOTIFY API CREDS
credentials = json.load(open('spotify_auth/authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [35]:
result = sp.search('1422', limit=1, offset=0, type='artist', market=None)
result['artists']['items'][0]['id']


'5tRm2pP1SxI1Px4VzSSwWZ'